<h1 align='center'>Classificação de spam Machine Learning | Implantação de ponta a ponta | classificador de spam usando Python</h1>

Data Scientist jr: Karina Gonçalves Soares

Links de estudo:

* [Repositório_Spam_classification](https://github.com/EddyGiusepe/Spam_classifier_using_Python/blob/main/spam_classification_ML_End_to_End_Deployment.ipynb)
* [Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)
